### Disclaimer
TERADATA DISCLAIMS ALL WARRANTIES RELATING TO THE CODE, EXPRESS OR IMPLIED, INCLUDING, BUT NOT LIMITED TO, ANY WARRANTIES AGAINST INFRINGEMENT OF THIRD-PARTY RIGHTS, MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE.

TERADATA SHALL NOT BE RESPONSIBLE OR LIABLE WITH RESPECT TO ANY SUBJECT MATTER OF THE CODE UNDER ANY CONTRACT, NEGLIGENCE, STRICT LIABILITY OR OTHER THEORY 
    (A) FOR LOSS OR INACCURACY OF DATA OR COST OF PROCUREMENT OF SUBSTITUTE GOODS, SERVICES OR TECHNOLOGY, OR 
    (B) FOR ANY INDIRECT, INCIDENTAL OR CONSEQUENTIAL DAMAGES INCLUDING, BUT NOT LIMITED TO LOSS OF REVENUES AND LOSS OF PROFITS. TERADATA SHALL NOT BE RESPONSIBLE FOR ANY MATTER BEYOND ITS REASONABLE CONTROL.

Notwithstanding anything to the contrary: 
    (a) Teradata will have no obligation of any kind with respect to any Code-related comments, suggestions, design changes or improvements that you elect to provide to Teradata in either verbal or written form (collectively, “Feedback”), and 
    (b) Teradata and its affiliates are hereby free to use any ideas, concepts, know-how or techniques, in whole or in part, contained in Feedback: 
        (i) for any purpose whatsoever, including developing, manufacturing, and/or marketing products and/or services incorporating Feedback in whole or in part, and 
        (ii) without any restrictions or limitations, including requiring the payment of any license fees, royalties, or other consideration. 

## Introduction
##### User can create a GeoDataFrame on a Teradata table and view containing geospatial data. This allows user to retrieve, manipulate, process, and analyze geospatial information stored in the table with help of the GeoDataFrame Methods and GeoDataFrame Properties of teradataml GeoDataFrame.
If GeoDataFrame is created on a table that does not contain geometry data, then exception is raised.
teradataml GeoDataFrame extends the teradataml DataFrame that allows user to access and use teradataml DataFrame properties and methods.
Methods and Properties of teradataml GeoDataFrame will return teradataml GeoDataFrame, if the result contains geometry data. If result does not contain geometry data, then it returns teradataml DataFrame.

This notebook covers following:
1. Import the required teradataml modules.
2. Connect to a Vantage system.
3. Data loading.
4. Create a GeoDataFrame. 
5. Different GeoDataFrame properties and methods. 
6. Cleanup

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>1. Import the required modules.</b>

In [1]:
# Import.
import os
import getpass

from collections import OrderedDict

from teradataml.context.context import *
from teradataml import db_drop_table, read_csv
from teradataml import Point, LineString, Polygon, GeometryCollection, GeoSequence
from teradataml.geospatial.geodataframe import GeoDataFrame
from teradataml.utils.utils import execute_sql
from teradatasqlalchemy import (FLOAT, VARCHAR, INTEGER)

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>2. Connect to Vantage</b>
<p style = 'font-size:16px;font-family:Arial'>You will be prompted to enter the host name, username and password.</p>

In [2]:
con = create_context(host=getpass.getpass("Hostname: "), 
                     username=getpass.getpass("Username: "),
                     password=getpass.getpass("Password: "))

Hostname:  ········
Username:  ········
Password:  ········


<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>3. Data Loading</b>

In [3]:
execute_sql("CREATE TABLE customers(pkey integer, cust_name VARCHAR(10),cust_zipcode VARCHAR(5), location ST_Geometry);")
execute_sql("INSERT INTO customers(0, 'Fred Smith', '91234', new ST_Geometry('POINT(20.34 80.78)'));")
execute_sql("INSERT INTO customers(1, 'Richard Connor', '95421', new ST_Geometry('POINT(20.56 89.44)'));")
execute_sql("INSERT INTO customers(2, 'Anita Johnson', '98523', new ST_Geometry('POINT(19.12 88.22)'));")
execute_sql("INSERT INTO customers(3, 'Mary Willow', '90642', new ST_Geometry('POINT(21.45 88.12)'));")

TeradataCursor uRowsHandle=13 bClosed=False

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>4. Create a GeoDataFrame</b>

In [4]:
gdf = GeoDataFrame.from_table("customers")

In [5]:
# Print the Geo dataframe.
gdf

pkey,cust_name,cust_zipcode,location
2,Anita John,98523,POINT (19.12 88.22)
0,Fred Smith,91234,POINT (20.34 80.78)
1,Richard Co,95421,POINT (20.56 89.44)
3,Mary Willo,90642,POINT (21.45 88.12)


In [6]:
# Check the Teradata types for GeoDataFrame. Notice 'loc' column that is of type Geometry(). 
gdf.tdtypes

COLUMN NAME,TYPE
pkey,INTEGER()
cust_name,"VARCHAR(length=10, charset='LATIN')"
cust_zipcode,"VARCHAR(length=5, charset='LATIN')"
location,GEOMETRY()


In [7]:
# Check the Python types for GeoDataFrame.
gdf.dtypes

COLUMN NAME,TYPE
pkey,int
cust_name,str
cust_zipcode,str
location,str


<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>5. GeoDataFrame methods and properties</b> 

In [8]:
# select few columns from the GeoDataFrame. 
gdf.select(["pkey", "location"])

pkey,location
2,POINT (19.12 88.22)
0,POINT (20.34 80.78)
1,POINT (20.56 89.44)
3,POINT (21.45 88.12)


In [9]:
# Filter rows from the GeoDataFrame. 
gdf_filter = gdf[gdf.pkey.isin([1, 2])]
gdf_filter

pkey,cust_name,cust_zipcode,location
2,Anita John,98523,POINT (19.12 88.22)
1,Richard Co,95421,POINT (20.56 89.44)


In [10]:
# Check if a Geometry value has no anomalous geometric points, such as self intersection tangency.
# Resultant column "is_simple_location_geom" contains:
#    * 1, if the geometry is simple, with no anomalous points
#    * 0, if the geometry is not simple
is_simple = gdf.is_simple
is_simple

pkey,cust_name,cust_zipcode,location,is_simple_location_geom
2,Anita John,98523,POINT (19.12 88.22),1
0,Fred Smith,91234,POINT (20.34 80.78),1
1,Richard Co,95421,POINT (20.56 89.44),1
3,Mary Willo,90642,POINT (21.45 88.12),1


In [11]:
# Check if a Geometry value is well-formed.
# Resultant column "is_valid_location_geom" contains:
#    * 1, if the geometry is simple, with no anomalous points
#    * 0, if the geometry is not simple
is_valid = gdf.is_valid
is_valid

pkey,cust_name,cust_zipcode,location,is_valid_location_geom
1,Richard Co,95421,POINT (20.56 89.44),1
2,Anita John,98523,POINT (19.12 88.22),1
3,Mary Willo,90642,POINT (21.45 88.12),1
0,Fred Smith,91234,POINT (20.34 80.78),1


In [12]:
# Get the geometry type.
geom_type = gdf.geometry.geom_type

In [13]:
# Assign new columns. 
gdf.assign(geom_type = geom_type, is_simple = gdf.geometry.is_simple, is_valid = gdf.geometry.is_valid)

pkey,cust_name,cust_zipcode,location,geom_type,is_simple,is_valid
2,Anita John,98523,POINT (19.12 88.22),ST_Point,1,1
0,Fred Smith,91234,POINT (20.34 80.78),ST_Point,1,1
1,Richard Co,95421,POINT (20.56 89.44),ST_Point,1,1
3,Mary Willo,90642,POINT (21.45 88.12),ST_Point,1,1


In [14]:
# Find the x-coordinate. 
gdf.x

pkey,cust_name,cust_zipcode,location,x_location_geom
2,Anita John,98523,POINT (19.12 88.22),19.12
0,Fred Smith,91234,POINT (20.34 80.78),20.34
1,Richard Co,95421,POINT (20.56 89.44),20.56
3,Mary Willo,90642,POINT (21.45 88.12),21.45


In [15]:
# Find the y-coordinate. 
gdf.y

pkey,cust_name,cust_zipcode,location,y_location_geom
1,Richard Co,95421,POINT (20.56 89.44),89.44
2,Anita John,98523,POINT (19.12 88.22),88.22
3,Mary Willo,90642,POINT (21.45 88.12),88.12
0,Fred Smith,91234,POINT (20.34 80.78),80.78


In [16]:
# Print the column name that contains geometry data.
gdf.geometry.name

'location'

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>6. Cleanup</b>

In [17]:
# Drop the table. 
db_drop_table('customers')

True

In [18]:
# Remove context.
remove_context()

True